## 亂數設定
為了實現模型再現性，必須鎖定亂數使得模型每次訓練結果都一樣。

In [117]:
# 設定亂數種子數值
seed_value= 4
# 1. 設定 Python 環境變數亂數種子
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. 設定 Python 內建亂數生成器亂數種子
import random
random.seed(seed_value)
# 3. 設定 Numpy 亂數種子
import numpy as np
np.random.seed(seed_value)
# 4. 設定 TensorFlow 亂數種子
import tensorflow as tf
tf.random.set_seed(seed_value)

## 載入資料集

In [118]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# 載入 sklearn 房價預測資料集 13個輸入特徵 1個輸出特徵
boston_dataset = load_boston()
# 輸入特徵共13個
X = boston_dataset.data
# 輸出(房價)
y = boston_dataset.target

# 資料正規化
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

# 觀察切割後資料的維度與筆數
print(f'The shape of train_x: {X_train.shape}\t train_y: {y_train.shape}')
print(f'The shape of valid_x: {X_test.shape}\t valid_y: {y_test.shape}')

The shape of train_x: (404, 13)	 train_y: (404,)
The shape of valid_x: (102, 13)	 valid_y: (102,)


## 線性迴歸(Lasso Regression)
Lasso 是一個 linear model 再加上一個 L1-norm 的正規化所形成的模型(有重要特徵篩選的效果)。

Parameters:
- fit_intercept: 是否有截距，如果沒有則直線過原點。
- alpha: 調整 L1 正規化的限制式。可以調整的範圍：0~inf。
- positive: 如果 True 係數則為正，False 為負。預設為 True。

Attributes:
- coef_: 取得係數。
- intercept_: 取得截距。

Methods:
- fit: 放入X、y進行模型擬合。
- predict: 預測並回傳預測類別。
- score: R2 score 模型評估。


In [119]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

print(tf.__version__)

2.3.1


In [120]:
# 此範例使用 Tensorflow2.0 Sequential API 搭建神經網路。
def build_model():
    model = Sequential()
    model.add(Dense(128, Activation('relu'), input_dim=X.shape[-1]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))

    return model

In [121]:
tf.keras.backend.clear_session() # 清空網路暫存
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1792      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 16,321
Trainable params: 16,321
Non-trainable params: 0
_________________________________________________________________


In [122]:
# 編譯模型
optim = Adam(lr=0.001)
model.compile(loss='mse',
              optimizer=optim)

batch_size=16
epochs = 10

# 訓練模型
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    shuffle=True,
                    validation_split=0.1)#, callbacks=[lr_schedule])


Epoch 1/10
23/23 [==============================] - 0s 5ms/step - loss: 525.8920 - val_loss: 396.1129
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 203.6993 - val_loss: 86.1573
Epoch 3/10
23/23 [==============================] - 0s 2ms/step - loss: 68.6531 - val_loss: 28.1450
Epoch 4/10
23/23 [==============================] - 0s 2ms/step - loss: 30.3872 - val_loss: 22.6032
Epoch 5/10
23/23 [==============================] - 0s 2ms/step - loss: 24.0079 - val_loss: 16.9995
Epoch 6/10
23/23 [==============================] - 0s 2ms/step - loss: 19.5215 - val_loss: 14.0999
Epoch 7/10
23/23 [==============================] - 0s 2ms/step - loss: 17.0294 - val_loss: 14.2812
Epoch 8/10
23/23 [==============================] - 0s 2ms/step - loss: 16.0312 - val_loss: 14.0549
Epoch 9/10
23/23 [==============================] - 0s 2ms/step - loss: 14.6971 - val_loss: 12.6586
Epoch 10/10
23/23 [==============================] - 0s 2ms/step - loss: 13.5009 - val_loss: 12.6

In [123]:
from sklearn.metrics import mean_squared_error

print("訓練集")
y_pred = model.predict(X_train)
print('MSE:', mean_squared_error(y_pred, y_train))
print("測試集")
y_pred = model.predict(X_test)
print('MSE:', mean_squared_error(y_pred, y_test))

訓練集
MSE: 12.43095584219719
測試集
MSE: 18.034715732773478
